In [ ]:
include("./ROIontology.jl")


ROIexp = load("./Data/ROIexp.jld2", "ROIexp");
lfp = ROIexp[:lfp_data]
events = ROIexp[:behavioral_events]
trials = ROIexp[:trials]
sessions = ROIexp[:sessions]

include("./Links.jl")

const WD = filter(x -> window_data_event(window_data(x)) ∈ dom(event_trial), collect(dom(window_data)))
const WC = filter(x -> window_data_event(x) ∈ dom(event_trial), collect(codom(window_data)))

In [ ]:

using FourierAnalysis

# WT = event_trial∘window_data_event
# using JLD2 
# @save "TempData/event_lfp_windows_1_1.jld2"  WC
#split events into pre-peri-post with 512 sample pre and post windows
#events shorter than 512 are expanded
# wd_pre = GMap(x -> x[498:1010], WC)
wd_peri = GMap(x -> length(x) <= 2532 ? x[round(Int, length(x) / 2)-256:round(Int, length(x) / 2)+256] : x[1010:end-1010], WC)
# wd_post = GMap(x -> x[end-1010:end-498], WC)

# #Freq Dom
# # pre_spec = GMap(x -> spectra(x, 256, 512), wd_pre.(WC))
peri_spec = GMap(x -> spectra(x, 256, 512), wd_peri.(WC))
# # post_spec = GMap(x -> spectra(x, 256, 512), wd_post.(WC)) 

using DataFrames, DataFramesMeta

event_spectra = DataFrame()

event_spectra.rat = map(x -> window_lfp(inv(window_data)(x)).rat.name, WC)
event_spectra.region = map(x -> window_data_region(x).name, WC)
event_spectra.behavior_type = map(x -> window_data_event(x).behavior.name, WC)
event_spectra.trial_type = map(x -> event_trial(window_data_event(x)).condition.name, WC)
event_spectra.agent_type = map(x -> agenttype(event_trial(window_data_event(x)).condition), WC)
specs = map(x -> spectra(x, 256, 512), WC)
event_spectra.respiratory = map(x -> mean(x, (3, 12) .* 256 ./1010.1), specs)
event_spectra.theta = map(x -> mean(x, (5, 10) .* 256 ./1010.1), specs)
event_spectra.beta = map(x -> mean(x, (15, 35) .* 256 ./1010.1), specs)
event_spectra.gamma_low = map(x -> mean(x, (50, 59) .* 256 ./1010.1), specs)
event_spectra.gamma_high = map(x -> mean(x, (70, 100) .* 256 ./1010.1), specs)

esnull = copy(event_spectra)
esnull.behavior_type = fill("null", length(esnull.behavior_type))

event_spectra = vcat(event_spectra, esnull)


In [72]:
using MixedModels, ProgressMeter
ProgressMeter.ijulia_behavior(:append)

IJuliaAppend::IJuliaBehavior = 2

1) It is also hypothesized that hippocampal theta oscillation will increase in amplitude during rearing, due to theta being an exploration related sensorimotor rhythm. This difference should also be present in amygdala (though may be due to volume conduction).

In [73]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["null","Rear"]), :region.=="Ca2")
fm=@formula(theta ~ 1 + behavior_type*1 + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

|                     |   Est. |     SE |     z |      p |  σ_rat |
|:------------------- | ------:| ------:| -----:| ------:| ------:|
| (Intercept)         | 0.0051 | 0.0003 | 15.81 | <1e-55 | 0.0006 |
| behavior_type: Rear | 0.0004 | 0.0001 |  6.65 | <1e-10 |        |
| Residual            | 0.0017 |        |       |        |        |


In [74]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["null","Rear"]), :region.=="Amygdala")
fm=@formula(theta ~ 1 + behavior_type*1 + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

|                     |   Est. |     SE |    z |      p |  σ_rat |
|:------------------- | ------:| ------:| ----:| ------:| ------:|
| (Intercept)         | 0.0014 | 0.0002 | 8.16 | <1e-15 | 0.0004 |
| behavior_type: Rear | 0.0001 | 0.0000 | 5.75 | <1e-08 |        |
| Residual            | 0.0003 |        |      |        |        |


2. We expect that beta rhythms will be present when the animal is rearing, and not present during immobility. 

In [75]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref([ "null","Rear"]))
fm=@formula(beta ~0+region+ behavior_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

|                                        |   Est. |     SE |     z |      p |  σ_rat |
|:-------------------------------------- | ------:| ------:| -----:| ------:| ------:|
| region: Amygdala                       | 0.0002 | 0.0000 | 13.12 | <1e-38 | 0.0000 |
| region: Ca2                            | 0.0004 | 0.0000 |  9.41 | <1e-20 | 0.0001 |
| region: Insula                         | 0.0001 | 0.0000 |  7.05 | <1e-11 | 0.0000 |
| region: MOB                            | 0.0010 | 0.0002 |  5.05 | <1e-06 | 0.0004 |
| behavior_type: Rear & region: Amygdala | 0.0000 | 0.0000 |  0.80 | 0.4214 |        |
| behavior_type: Rear & region: Ca2      | 0.0000 | 0.0000 |  3.83 | 0.0001 |        |
| behavior_type: Rear & region: Insula   | 0.0000 | 0.0000 |  0.08 | 0.9350 |        |
| behavior_type: Rear & region: MOB      | 0.0000 | 0.0000 |  1.10 | 0.2722 |        |
| Residual                               | 0.0002 |        |       |        |        |


3. It is hypothesized that theta will show increased amplitude during behaviors associated with exploration like rearing, when compared with regulatory behaviors like grooming and immobility. This means that rearing will be significant different from immobility/grooming in both respiratory and theta rhythms in MOB, CA2, and Amyg. 

In [76]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["Immobility","Groom","Rear"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm=@formula(theta ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Groom")))

|                                            |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: Amygdala                           | 0.0012 | 0.0002 |  7.89 | <1e-14 | 0.0002 |
| region: Ca2                                | 0.0041 | 0.0003 | 14.36 | <1e-46 | 0.0005 |
| region: Insula                             | 0.0010 | 0.0002 |  5.04 | <1e-06 | 0.0002 |
| region: MOB                                | 0.0083 | 0.0033 |  2.50 | 0.0126 | 0.0067 |
| combined_behavior: Rear & region: Amygdala | 0.0001 | 0.0001 |  0.99 | 0.3231 |        |
| combined_behavior: Rear & region: Ca2      | 0.0011 | 0.0001 |  7.48 | <1e-13 |        |
| combined_behavior: Rear & region: Insula   | 0.0002 | 0.0002 |  1.23 | 0.2173 |        |
| combined_behavior: Rear & region: MOB      | 0.0091 | 0.0002 | 53.45 | <1e-99 |        |
| Residual                                   | 0.0031 |        |       |        |        |


In [77]:
fm=@formula(respiratory ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Groom")))

|                                            |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------ | -------:| ------:| -----:| ------:| ------:|
| region: Amygdala                           |  0.0016 | 0.0002 | 10.24 | <1e-23 | 0.0002 |
| region: Ca2                                |  0.0040 | 0.0002 | 17.05 | <1e-64 | 0.0004 |
| region: Insula                             |  0.0012 | 0.0002 |  6.06 | <1e-08 | 0.0003 |
| region: MOB                                |  0.0104 | 0.0030 |  3.41 | 0.0006 | 0.0061 |
| combined_behavior: Rear & region: Amygdala | -0.0002 | 0.0001 | -1.93 | 0.0535 |        |
| combined_behavior: Rear & region: Ca2      |  0.0001 | 0.0001 |  1.30 | 0.1947 |        |
| combined_behavior: Rear & region: Insula   | -0.0002 | 0.0001 | -1.41 | 0.1589 |        |
| combined_behavior: Rear & region: MOB      |  0.0031 | 0.0001 | 23.44 | <1e-99 |        |
| Residual                                   |  0.0024 |        |       |        |        |


4. It is hypothesized that robot will elicit a higher low-gamma rhythm in MOB for the robot condition when compared with the object and rat conditions for immobility and grooming behaviors. 


In [78]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat & Object"
    elseif atype .== Object
        return "Rat & Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :combined_agent = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + combined_agent & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_agent => DummyCoding(; base = "Rat & Object")))

|                                                   |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Groom                              |  0.0016 | 0.0002 |  7.61 | <1e-13 |        |
| behavior_type: Immobility                         |  0.0007 | 0.0002 |  3.02 | 0.0026 |        |
| behavior_type: Rear                               |  0.0004 | 0.0002 |  1.86 | 0.0622 |        |
| behavior_type: null                               |  0.0006 | 0.0002 |  3.26 | 0.0011 |        |
| combined_agent: Robot & behavior_type: Groom      | -0.0003 | 0.0001 | -2.22 | 0.0263 |        |
| combined_agent: Robot & behavior_type: Immobility | -0.0000 | 0.0001 | -0.03 | 0.9744 |        |
| combined_agent: Robot & behavior_type: Rear       | -0.0000 | 0.0001 | -0.04 | 0.9715 |        |
| combined_agent: Robot & behavior_type: null       | -0.0000 | 0.0000 | -0.96 | 0.3380 |        |
| (Intercept)                                       |         |        |       |        | 0.0004 |
| Residual                                          |  0.0008 |        |       |        |        |


In [79]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat & Object"
    elseif atype .== Object
        return "Rat & Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :combined_agent = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + combined_agent & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_agent => DummyCoding(; base = "Rat & Object")))

|                                                   |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Groom                              |  0.0016 | 0.0002 |  7.61 | <1e-13 |        |
| behavior_type: Immobility                         |  0.0007 | 0.0002 |  3.02 | 0.0026 |        |
| behavior_type: Rear                               |  0.0004 | 0.0002 |  1.86 | 0.0622 |        |
| behavior_type: null                               |  0.0006 | 0.0002 |  3.26 | 0.0011 |        |
| combined_agent: Robot & behavior_type: Groom      | -0.0003 | 0.0001 | -2.22 | 0.0263 |        |
| combined_agent: Robot & behavior_type: Immobility | -0.0000 | 0.0001 | -0.03 | 0.9744 |        |
| combined_agent: Robot & behavior_type: Rear       | -0.0000 | 0.0001 | -0.04 | 0.9715 |        |
| combined_agent: Robot & behavior_type: null       | -0.0000 | 0.0000 | -0.96 | 0.3380 |        |
| (Intercept)                                       |         |        |       |        | 0.0004 |
| Residual                                          |  0.0008 |        |       |        |        |


5. High gamma amplitude should be highest for the rat condition in the MOB. The robot and object should have similar high gamma. But instead, it appears that the robot and rat high gamma is larger than the object, and there is no discernable difference between rat and robot high gamma in MOB. 

In [80]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_high ~ 0 + behavior_type + agent_type & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))


|                                                |    Est. |     SE |     z |      p |  σ_rat |
|:---------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Groom                           |  0.0005 | 0.0001 |  4.87 | <1e-05 |        |
| behavior_type: Immobility                      |  0.0004 | 0.0001 |  3.89 | 0.0001 |        |
| behavior_type: Rear                            |  0.0004 | 0.0001 |  4.31 | <1e-04 |        |
| behavior_type: null                            |  0.0006 | 0.0001 |  5.69 | <1e-07 |        |
| agent_type: Object & behavior_type: Groom      | -0.0000 | 0.0001 | -0.78 | 0.4355 |        |
| agent_type: Robot & behavior_type: Groom       | -0.0000 | 0.0000 | -0.39 | 0.6963 |        |
| agent_type: Object & behavior_type: Immobility |  0.0000 | 0.0001 |  0.44 | 0.6609 |        |
| agent_type: Robot & behavior_type: Immobility  | -0.0000 | 0.0001 | -0.03 | 0.9756 |        |
| agent_type: Object & behavior_type: Rear       | -0.0000 | 0.0000 | -0.19 | 0.8522 |        |
| agent_type: Robot & behavior_type: Rear        |  0.0001 | 0.0000 |  1.29 | 0.1972 |        |
| agent_type: Object & behavior_type: null       | -0.0001 | 0.0000 | -3.42 | 0.0006 |        |
| agent_type: Robot & behavior_type: null        | -0.0001 | 0.0000 | -5.40 | <1e-07 |        |
| (Intercept)                                    |         |        |       |        | 0.0002 |
| Residual                                       |  0.0003 |        |       |        |        |


In [81]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + agent_type & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Robot")))


|                                                |    Est. |     SE |     z |      p |  σ_rat |
|:---------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Groom                           |  0.0013 | 0.0002 |  5.72 | <1e-07 |        |
| behavior_type: Immobility                      |  0.0007 | 0.0002 |  3.21 | 0.0013 |        |
| behavior_type: Rear                            |  0.0004 | 0.0002 |  1.79 | 0.0730 |        |
| behavior_type: null                            |  0.0006 | 0.0002 |  3.05 | 0.0023 |        |
| agent_type: Object & behavior_type: Groom      |  0.0004 | 0.0002 |  2.44 | 0.0146 |        |
| agent_type: Rat & behavior_type: Groom         |  0.0002 | 0.0001 |  1.76 | 0.0781 |        |
| agent_type: Object & behavior_type: Immobility |  0.0000 | 0.0002 |  0.13 | 0.8969 |        |
| agent_type: Rat & behavior_type: Immobility    | -0.0000 | 0.0002 | -0.06 | 0.9516 |        |
| agent_type: Object & behavior_type: Rear       |  0.0001 | 0.0001 |  0.41 | 0.6841 |        |
| agent_type: Rat & behavior_type: Rear          | -0.0000 | 0.0001 | -0.20 | 0.8423 |        |
| agent_type: Object & behavior_type: null       |  0.0001 | 0.0001 |  2.13 | 0.0330 |        |
| agent_type: Rat & behavior_type: null          |  0.0000 | 0.0000 |  0.24 | 0.8098 |        |
| (Intercept)                                    |         |        |       |        | 0.0004 |
| Residual                                       |  0.0008 |        |       |        |        |


6. It is hypothesized that the amygdala beta rhythm shows a difference between grooming/immobility and rearing. It would be expected that immobility shows the least beta, with grooming showing some due to arm movements, and rearing showing the most. 

In [82]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "Rear"]), :region .== "Amygdala")
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(beta ~ 1 + combined_behavior * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "Immobility & Groom")))

|                         |    Est. |     SE |     z |      p |  σ_rat |
|:----------------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)             |  0.0002 | 0.0000 | 15.64 | <1e-54 | 0.0000 |
| combined_behavior: Rear | -0.0000 | 0.0000 | -0.95 | 0.3441 |        |
| Residual                |  0.0001 |        |       |        |        |


7.  For regulatory behaviors (grooming/immobility) there should be a shift to lower frequencies within the respiratory range and more low gamma, while exploratory behaviors should shift to theta frequency and high gamma (rearing, approach/retreat). 

In [83]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "null"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(theta ~ 0 + region + combined_behavior & region + zerocorr(0 + region | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "null")))

|                                                          |    Est. |     SE |      z |      p |  σ_rat |
|:-------------------------------------------------------- | -------:| ------:| ------:| ------:| ------:|
| region: Amygdala                                         |  0.0014 | 0.0002 |   8.94 | <1e-18 | 0.0003 |
| region: Ca2                                              |  0.0051 | 0.0004 |  14.18 | <1e-44 | 0.0007 |
| region: Insula                                           |  0.0011 | 0.0002 |   6.96 | <1e-11 | 0.0002 |
| region: MOB                                              |  0.0145 | 0.0032 |   4.50 | <1e-05 | 0.0064 |
| combined_behavior: Immobility & Groom & region: Amygdala | -0.0001 | 0.0001 |  -0.55 | 0.5847 |        |
| combined_behavior: Immobility & Groom & region: Ca2      | -0.0007 | 0.0001 |  -5.15 | <1e-06 |        |
| combined_behavior: Immobility & Groom & region: Insula   | -0.0001 | 0.0001 |  -0.65 | 0.5132 |        |
| combined_behavior: Immobility & Groom & region: MOB      | -0.0059 | 0.0002 | -36.80 | <1e-99 |        |
| Residual                                                 |  0.0035 |        |        |        |        |


In [84]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "null"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(gamma_high ~ 0 + region + combined_behavior & region + zerocorr(0 + region | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "null")))

|                                                          |    Est. |     SE |     z |      p |  σ_rat |
|:-------------------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| region: Amygdala                                         |  0.0001 | 0.0000 |  3.48 | 0.0005 | 0.0000 |
| region: Ca2                                              |  0.0000 | 0.0000 | 18.01 | <1e-71 | 0.0000 |
| region: Insula                                           |  0.0000 | 0.0000 |  5.29 | <1e-06 | 0.0000 |
| region: MOB                                              |  0.0005 | 0.0001 |  5.21 | <1e-06 | 0.0002 |
| combined_behavior: Immobility & Groom & region: Amygdala |  0.0000 | 0.0000 |  0.17 | 0.8643 |        |
| combined_behavior: Immobility & Groom & region: Ca2      | -0.0000 | 0.0000 | -0.20 | 0.8416 |        |
| combined_behavior: Immobility & Groom & region: Insula   |  0.0000 | 0.0000 |  0.26 | 0.7963 |        |
| combined_behavior: Immobility & Groom & region: MOB      | -0.0000 | 0.0000 | -2.34 | 0.0195 |        |
| Residual                                                 |  0.0001 |        |       |        |        |


In [85]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :behavior_type .∈ Ref(["Immobility"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm=@formula(theta ~0+region+ agent_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

|                                       |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| region: Amygdala                      |  0.0014 | 0.0006 |  2.23 | 0.0256 | 0.0000 |
| region: Ca2                           |  0.0035 | 0.0008 |  4.56 | <1e-05 | 0.0006 |
| region: Insula                        |  0.0016 | 0.0006 |  2.60 | 0.0093 | 0.0000 |
| region: MOB                           |  0.0131 | 0.0031 |  4.22 | <1e-04 | 0.0052 |
| agent_type: Object & region: Amygdala | -0.0003 | 0.0010 | -0.32 | 0.7493 |        |
| agent_type: Robot & region: Amygdala  | -0.0001 | 0.0007 | -0.19 | 0.8468 |        |
| agent_type: Object & region: Ca2      |  0.0003 | 0.0012 |  0.20 | 0.8382 |        |
| agent_type: Robot & region: Ca2       |  0.0007 | 0.0008 |  0.85 | 0.3980 |        |
| agent_type: Object & region: Insula   | -0.0005 | 0.0012 | -0.38 | 0.7048 |        |
| agent_type: Robot & region: Insula    | -0.0005 | 0.0008 | -0.62 | 0.5348 |        |
| agent_type: Object & region: MOB      |  0.0023 | 0.0011 |  2.07 | 0.0388 |        |
| agent_type: Robot & region: MOB       |  0.0034 | 0.0009 |  3.94 | <1e-04 |        |
| Residual                              |  0.0039 |        |       |        |        |
